Importing required libraries

In [ ]:
import streamlit as st
from streamlit_chat import message
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import CTransformers
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory

Create a function to load documents

In [ ]:
def load_documents():
    loader=DirectoryLoader('data/',glob="*.pdf",loader_cls=PyPDFLoader)
    documents= loader.load()
    return documents

Function to split text into chunks

In [ ]:
def split_text_into_chunks(documents):
    text_splitter= RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    text_chunks=text_splitter.split_documents(documents)
    return text_chunks

Function to create embdeddings

In [ ]:
def create_embdeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",model_kwargs={'device':"cpu"})
    return embeddings

Function to create vector store

In [ ]:
def create_vector_store(text_chunks,embeddings):
    vector_store=FAISS.from_documents(text_chunks,embeddings)
    return vector_store

Function to create LLMS Model

In [ ]:
def create_llms_model():
    llm=CTransformers(model="TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
                      config={'max_new_tokens':128,
                              'temperature':0.01},
                              streaming=True)
    return llm

Initialize Streamlit app

In [ ]:
st.title("Job Interview Prep Chatbot")
st.title("Personalised Job Success Friend")
st.markdown('<style>h1{color:orange;text-align:center;}</style>',unsafe_allow_html=True)
st.subheader("Get your desired Job !")
st.markdown('<style>h3{color:pink;text-align:center;}</style>',unsafe_allow_html=True)

loading of documents

In [ ]:
documents=load_documents()

Split text into chunks

In [ ]:
text_chunks=split_text_into_chunks(documents)

Create embeddings

In [ ]:
embeddings=create_embdeddings()

Create Vector Store

In [ ]:
vector_store=create_vector_store(text_chunks,embeddings)

Create LLMS model

In [ ]:
llm=create_llms_model()

Initialize the conversation history

In [ ]:
if 'history' not in st.session_state:
    st.session_state['history'] = []

if 'generated' not in st.session_state:
    st.session_state['generated'] = ["Hello! Ask me anything about your dream job!"]

if 'past' not in st.session_state:
    st.session_state['past'] = ["Hey! Let's have a fruitful chat"]

Create memory

In [ ]:
memory= ConversationBufferMemory(memory_key="chat_history",return_messages=True)

Create chain

In [ ]:
chain = ConversationalRetrievalChain.from_llm(llm=llm,chain_type='stuff',
                                              retriever=vector_store.as_retriever(search_kwargs={"k":2}),
                                              memory=memory)


Define chat function

In [ ]:
def conversation_chat(query):
    result=chain({"question":query,"chat_history":st.session_state['history']})
    st.session_state['history'].append((query,result["answer"]))
    return result["answer"]

Display chat history

In [ ]:
reply_container=st.container()
container=st.container()

with container:
    with st.form(key="my form",clear_on_submit=True):
        user_input=st.text_input("Question:",placeholder="Ask about your Job Interview", key='input')
        submit_button=st.form_submit_button(label='send')
    
    if submit_button and user_input:
        output= conversation_chat(user_input)
        st.session_state['past'].append(user_input)
        st.session_state['generated'].append(output)

if st.session_state['generated']:
    with reply_container:
        for i in range(len(st.session_state['generated'])):
            message(st.session_state["past"][i],is_user=True,key=str(i)+'_user',avatar_style="thumbs")
            message(st.session_state["generated"][i],key=str(i),avatar_style="fun-emoji")